In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


numerical = pd.read_csv('numerical.csv')
categorical = pd.read_csv('categorical.csv')
targets = pd.read_csv('target.csv')
data = pd.concat([numerical, categorical, targets], axis = 1)
data['TARGET_B'].value_counts()

In [ ]:
data.head()

In [ ]:
data.isna().sum().sum()

In [ ]:
data.columns

In [ ]:
data[data['FIRSTDATE_MM'].isna()]

In [ ]:
numerical.shape

In [ ]:
data.head()

In [ ]:
# Importing the necessary libraries
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import resample




In [ ]:
y = data['TARGET_B']
X = data.drop(['TARGET_B','TARGET_D'], axis = 1)

numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(np.object)

# we OneHotEncode the categoricals so we can use the same dataset to perform a regression later (in the lab).
# it is not needed for a DecisionTree or RandomForest model
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(categoricalX)
encoded_categorical = encoder.transform(categoricalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)
X = pd.concat([numericalX, encoded_categorical], axis = 1)

# Note: we need to do train/test split before upsampling, and then only upsample the training set - Why?
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
#Upsampling


# for UPnsampling we need to temporarily concat X_train and y_train
trainset = pd.concat([X_train, y_train], axis=1)
# quicker way to downsample category 0:
category_1 = trainset[trainset['TARGET_B']== 1].sample(len(trainset[trainset['TARGET_B']==0]),replace=True)
print(category_1.shape)

category_0 = trainset[trainset['TARGET_B']== 0 ]
trainset_new = pd.concat([category_0, category_1], axis = 0)
trainset_new = trainset_new.sample(frac =1) #randomize the rows
X_train = trainset_new.drop(['TARGET_B'], axis=1)
y_train = trainset_new['TARGET_B']
#data = data.reset_index(drop=True)
print(X_train.shape)

In [ ]:
y_train.shape

In [ ]:
y_test.value_counts()

In [ ]:
y_train.value_counts()

In [ ]:
X.columns = [str(c) for c in X.columns]


In [ ]:
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.2,
                             random_state = 42)
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score



In [ ]:
clf = RandomForestClassifier(max_depth=10,
                             min_samples_split=20,
                             min_samples_leaf=20).fit(X_train, y_train)
pred_RF = clf.predict(X_test)

print('accuracy:', accuracy_score(y_test, pred_RF))
print("precision: ", precision_score(y_test, pred_RF))
print("recall: ", recall_score(y_test, pred_RF))
print("f1: ", f1_score(y_test, pred_RF))


In [ ]:
# To check the scores
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

model = SelectKBest(chi2, k=10).fit(X, y)
df = pd.DataFrame(data= model.scores_, columns = ['score'])
df['Column'] = X.columns
print(df.sort_values(by = ['score'], ascending = False).head(10))

In [ ]:
cols = df.sort_values(by = ['score'], ascending = False).head(30)['Column']
cols

In [ ]:
X_train=X_train[cols]
X_test=X_test[cols]

In [ ]:
lf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.2,
                             random_state = 42)
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

## Lab | Final regression model in "Health Care for All" Case


In [ ]:
#define new data fram that only includes rows where TARGET_B is equal to 1:
df_donors = data[data['TARGET_B']==1]
X= df_donors.drop(columns=['TARGET_B','TARGET_D'])
y = df_donors['TARGET_D']

In [ ]:
X.isna().sum().sum()

In [ ]:
y.isna().sum().sum()

In [ ]:

# Note: we need to do train/test split before upsampling, and then only upsample the training set - Why?
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train_num = X_train.select_dtypes(np.number)
X_train_cat = X_train.select_dtypes(np.object)
X_test_cat=X_test.select_dtypes(np.object)
X_test_num=X_test.select_dtypes(np.number)

# we OneHotEncode the categoricals so we can use the same dataset to perform a regression later (in the lab).
# it is not needed for a DecisionTree or RandomForest model
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(X_train_cat)
encoded_categorical = encoder.transform(X_train_cat).toarray()
X_train_cat_encoded = pd.DataFrame(encoded_categorical)



In [ ]:
#encode test
encoded_categorical_1 = encoder.transform(X_test_cat).toarray()
X_test_cat_encoded = pd.DataFrame(encoded_categorical_1)

In [ ]:
#transform
from sklearn.preprocessing import MinMaxScaler
transformer = MinMaxScaler().fit(X_train_num)
X_train_norm = transformer.transform(X_train_num)
print(X_train_norm.shape)
X_train_num_scale = pd.DataFrame(X_train_norm, index = X_train_num.index, columns=X_train_num.columns)
X_train_num_scale.head()

In [ ]:
X_train_num_scale.isna().sum().sum()

In [ ]:
X_test_norm = transformer.transform(X_test_num)
print(X_test_norm.shape)
X_test_num_scale = pd.DataFrame(X_test_norm, index = X_test_num.index, columns=X_test_num.columns)
X_test_num_scale.head()

In [ ]:
X_test_num_scale.reset_index(drop=True, inplace=True)
X_test_cat_encoded.reset_index(drop=True, inplace=True)
X_test = pd.concat([X_test_num_scale, X_test_cat_encoded], axis=1)

In [ ]:
X_test_num_scale.shape

In [ ]:
 X_test_cat_encoded.shape

In [ ]:
X_test.head()

In [ ]:
X_test.isna().sum().sum()

In [ ]:
X_train.isna().sum().sum()

In [ ]:
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

In [ ]:
from sklearn.linear_model import LinearRegression

#Fit a linear regression model to the training data
model = LinearRegression()
model.fit(X_train, y_train)

# Use the model to make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the performance of the model using a metric such as mean squared error
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Mean squared error:", mse)